In [8]:
import datetime
import matplotlib
import os
import time
from matplotlib import style

matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.visualization import make_lupton_rgb
from pprint import pprint
from glob import glob
import pickle
from tqdm import tqdm

In [9]:
# !jupyter nbconvert --to python hlwas.ipynb --TemplateExporter.exclude_markdown=True --TemplateExporter.exclude_output_prompt=True --TemplateExporter.exclude_input_prompt=True

In [10]:
start = time.time()

In [11]:
import stips

print(stips.__env__report__)

STIPS Version 2.1.0 with Data Version 1.0.9 at /data/bwedig/STScI/ref_data/stips_data.
	STIPS Grid Generated with 1.0.8
Pandeia Version 3.0 with Data Version 3.0 at /data/bwedig/STScI/ref_data/pandeia_data-3.0_roman.
Webbpsf Version 1.1.1 with Data Version 1.1.1 at /data/bwedig/STScI/ref_data/webbpsf-data.



In [12]:
num_detectors = 1
detector_list = [f'SCA{str(i + 1).zfill(2)}' for i in range(num_detectors)]
bands = ['F106', 'F129', 'F184']  # 
exposure_time = 1000

obs_id = 1
obs_prefix = 'hlwas_sim'
obs_ra = 30.
obs_dec = -30.

output_dir = os.path.join('/data/bwedig/STIPS')

In [13]:
# clear out old files
for f in glob(os.path.join(output_dir, f'{obs_prefix}_*.fits')):
    os.remove(f)

In [14]:
from stips.scene_module import SceneModule

scm = SceneModule(out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec, out_path=output_dir)

stellar_parameters = {
    'n_stars': 1000,
    'age_low': 11e12,  # 7.5e12
    'age_high': 2e12,  # 7.5e12
    'z_low': -2.0,
    'z_high': -2.0,
    'imf': 'salpeter',
    'alpha': -2.35,
    'binary_fraction': 0.1,
    'clustered': False,
    'distribution': 'uniform',
    'radius': 275.,  # 100.
    'radius_units': 'arcsec',
    'distance_low': 1.,  # 10.0
    'distance_high': 10.0,  # 10.0
    'offset_ra': 0.0,
    'offset_dec': 0.0
}

stellar_cat_file = scm.CreatePopulation(stellar_parameters)
print("Stellar population saved to file {}".format(stellar_cat_file))

galaxy_cat_file = '/data/bwedig/STIPS/hlwas_sim_gals_000.fits'

# galaxy_parameters = {
#                      'n_gals': 10,
#                      'z_low': 0.0,
#                      'z_high': 0.5,  # 0.2
#                      'rad_low': 0.01,
#                      'rad_high': 2.0,  # 2.
#                      'sb_v_low': 25.0,  # 28
#                      'sb_v_high': 22.0,  # 24.
#                      'distribution': 'uniform',
#                      'clustered': False,
#                      'radius': 2800.0,
#                      'radius_units': 'arcsec',
#                      'offset_ra': 0.0,
#                      'offset_dec': 0.0,
#                     }

# galaxy_cat_file = scm.CreateGalaxies(galaxy_parameters)
# print("Galaxy population saved to file {}".format(galaxy_cat_file))

2024-04-10 13:36:00,724 INFO: Creating catalogue /data/bwedig/STIPS/test_stars_000.fits
2024-04-10 13:36:00,725 INFO: Creating age and metallicity numbers
2024-04-10 13:36:00,726 INFO: Created age and metallicity numbers
2024-04-10 13:36:00,727 INFO: Creating stars
2024-04-10 13:36:00,727 INFO: Age 1.35e+10
2024-04-10 13:36:00,728 INFO: Metallicity -2.000000
2024-04-10 13:36:00,729 INFO: Creating 1000 stars


Log level: INFO


2024-04-10 13:36:01,620 INFO: Creating 1000 objects, max radius 275.0, function uniform, scale 2.8
2024-04-10 13:36:01,768 INFO: Chunk 1: 1094 stars
2024-04-10 13:36:01,816 INFO: Done creating catalogue
2024-04-10 13:36:01,818 INFO: Creating catalogue /data/bwedig/STIPS/test_gals_000.fits
2024-04-10 13:36:01,818 INFO: Wrote preamble
2024-04-10 13:36:01,819 INFO: Parameters are: {'n_gals': 10, 'z_low': 0.0, 'z_high': 0.5, 'rad_low': 0.01, 'rad_high': 2.0, 'sb_v_low': 25.0, 'sb_v_high': 22.0, 'distribution': 'uniform', 'clustered': False, 'radius': 2800.0, 'radius_units': 'arcsec', 'offset_ra': 0.0, 'offset_dec': 0.0}
2024-04-10 13:36:01,821 INFO: Making Co-ordinates
2024-04-10 13:36:01,822 INFO: Creating 10 objects, max radius 2800.0, function uniform, scale 2.8
2024-04-10 13:36:01,823 INFO: Converting Co-ordinates into RA,DEC
2024-04-10 13:36:01,862 INFO: Done creating catalogue


Stellar population saved to file /data/bwedig/STIPS/test_stars_000.fits
Galaxy population saved to file /data/bwedig/STIPS/test_gals_000.fits


In [56]:
from stips.observation_module import ObservationModule

offset = {
    'offset_id': 1,
    'offset_centre': False,
    'offset_ra': 0,
    'offset_dec': 0,
    'offset_pa': 0.0
}

residuals = {
    'residual_flat': False,
    'residual_dark': False,
    'residual_cosmic': False,
    'residual_poisson': False,
    'residual_readnoise': False
}

observation_parameters = {
    'instrument': 'WFI',
    'filters': bands,
    'detectors': num_detectors,
    'distortion': False,
    'background': 0.15,
    'observations_id': obs_id,
    'exptime': exposure_time,
    'offsets': [offset]
}

obm = ObservationModule(observation_parameters, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec, residual=residuals,
                        out_path=output_dir)

2024-04-05 21:21:12,820 INFO: Got offsets as [{'offset_id': 1, 'offset_centre': False, 'offset_ra': 0, 'offset_dec': 0, 'offset_pa': 0.0}]
2024-04-05 21:21:12,845 INFO: Adding observation with filter F106 and offset (0,0,0.0)
2024-04-05 21:21:12,846 INFO: Adding observation with filter F129 and offset (0,0,0.0)
2024-04-05 21:21:12,846 INFO: Adding observation with filter F184 and offset (0,0,0.0)
2024-04-05 21:21:12,847 INFO: Added 3 observations


2024-04-05 21:21:13,049 INFO: WFI with 18 detectors. Central offset (2.065830864204697e-13, -8.945310041616143e-14, 0)


In [57]:
# determine number of observations
num_obs = len(observation_parameters['filters'])

for i in tqdm(range(num_obs)):  # num_obs
    obm.nextObservation()

    output_stellar_catalogues = obm.addCatalogue(stellar_cat_file)
    # output_galaxy_catalogues = obm.addCatalogue(galaxy_cat_file)

    # print("Output Catalogues are {} and {}".format(output_stellar_catalogues, output_galaxy_catalogues))

    obm.addError()

    fits_file, mosaic_file, params = obm.finalize(mosaic=False)

    print("Output FITS file is {}".format(fits_file))
    print("Output Mosaic file is {}".format(mosaic_file))
    print("Observation Parameters are {}".format(params))

  0%|          | 0/3 [00:00<?, ?it/s]2024-04-05 21:21:13,059 INFO: Initializing Observation 0 of 3
2024-04-05 21:21:13,059 INFO: Observation Filter is F106
2024-04-05 21:21:13,059 INFO: Observation (RA,DEC) = (30.0,-30.0) with PA=0.0
2024-04-05 21:21:13,060 INFO: Resetting
2024-04-05 21:21:13,061 INFO: Returning background 0.15.
2024-04-05 21:21:13,062 INFO: Creating Detector SCA01 with (RA,DEC,PA) = (30.0,-30.0,0.0)
2024-04-05 21:21:13,062 INFO: Creating Detector SCA01 with offset (0.0,0.0)
2024-04-05 21:21:13,088 INFO: Creating Instrument with Configuration {'aperture': 'imaging', 'disperser': None, 'filter': 'f106', 'instrument': 'wfi', 'mode': 'imaging'}
2024-04-05 21:21:13,160 INFO: SCA01: (RA, DEC, PA) := (30.0, -30.0, 0.0), detected as (30.0, -30.0, 0.0)
2024-04-05 21:21:13,161 INFO: Detector SCA01 created
2024-04-05 21:21:13,161 INFO: Creating Detector SCA02 with (RA,DEC,PA) = (30.000090051032178,-30.14636790526755,0.0)
2024-04-05 21:21:13,162 INFO: Creating Detector SCA02 with

2024-04-05 21:21:13,218 INFO: SCA02: (RA, DEC, PA) := (30.000090051032178, -30.14636790526755, 0.0), detected as (30.000090051032178, -30.14636790526755, 0.0)
2024-04-05 21:21:13,219 INFO: Detector SCA02 created
2024-04-05 21:21:13,219 INFO: Creating Detector SCA03 with (RA,DEC,PA) = (30.00024711458644,-30.277285221181245,0.0)
2024-04-05 21:21:13,220 INFO: Creating Detector SCA03 with offset (0.0002471145864395015,-0.2772852211812461)
2024-04-05 21:21:13,275 INFO: SCA03: (RA, DEC, PA) := (30.00024711458644, -30.277285221181245, 0.0), detected as (30.00024711458644, -30.277285221181245, 0.0)
2024-04-05 21:21:13,276 INFO: Detector SCA03 created
2024-04-05 21:21:13,276 INFO: Creating Detector SCA04 with (RA,DEC,PA) = (29.8433541331425,-29.972902151206657,0.0)
2024-04-05 21:21:13,277 INFO: Creating Detector SCA04 with offset (-0.15664586685750012,0.02709784879334294)


Creating pandeia instrument roman.wfi.imaging


2024-04-05 21:21:13,331 INFO: SCA04: (RA, DEC, PA) := (29.8433541331425, -29.972902151206657, 0.0), detected as (29.8433541331425, -29.972902151206657, 0.0)
2024-04-05 21:21:13,332 INFO: Detector SCA04 created
2024-04-05 21:21:13,333 INFO: Creating Detector SCA05 with (RA,DEC,PA) = (29.84327149478523,-30.119578781351112,0.0)
2024-04-05 21:21:13,333 INFO: Creating Detector SCA05 with offset (-0.15672850521477064,-0.11957878135111233)
2024-04-05 21:21:13,387 INFO: SCA05: (RA, DEC, PA) := (29.84327149478523, -30.119578781351112, 0.0), detected as (29.84327149478523, -30.119578781351112, 0.0)
2024-04-05 21:21:13,388 INFO: Detector SCA05 created
2024-04-05 21:21:13,389 INFO: Creating Detector SCA06 with (RA,DEC,PA) = (29.843451405244423,-30.250174749241296,0.0)
2024-04-05 21:21:13,389 INFO: Creating Detector SCA06 with offset (-0.15654859475557562,-0.2501747492412948)
2024-04-05 21:21:13,443 INFO: SCA06: (RA, DEC, PA) := (29.843451405244423, -30.250174749241296, 0.0), detected as (29.843451

Output FITS file is /data/bwedig/STIPS/hlwas_sim_1_0.fits
Output Mosaic file is None
Observation Parameters are ['Instrument: WFI', 'Filters: F106', 'Pixel Scale: (0.110,0.110) arcsec/pix', 'Pivot Wavelength: 1.060 um', 'Background Type: Set to 0.15', 'Exposure Time: 1000.0s', 'Input Unit: counts/s', 'Added Flatfield Residual: False', 'Added Dark Current Residual: False', 'Added Cosmic Ray Residual: False', 'Added Readnoise: False', 'Added Poisson Noise: False', 'Detector X size: 4088', 'Detector Y size: 4088', 'Random Seed: 0']


2024-04-05 21:21:32,065 INFO: SCA03: (RA, DEC, PA) := (30.00024711458644, -30.277285221181245, 0.0), detected as (30.00024711458644, -30.277285221181245, 0.0)
2024-04-05 21:21:32,066 INFO: Detector SCA03 created
2024-04-05 21:21:32,066 INFO: Creating Detector SCA04 with (RA,DEC,PA) = (29.8433541331425,-29.972902151206657,0.0)
2024-04-05 21:21:32,067 INFO: Creating Detector SCA04 with offset (-0.15664586685750012,0.02709784879334294)
2024-04-05 21:21:32,122 INFO: SCA04: (RA, DEC, PA) := (29.8433541331425, -29.972902151206657, 0.0), detected as (29.8433541331425, -29.972902151206657, 0.0)
2024-04-05 21:21:32,123 INFO: Detector SCA04 created
2024-04-05 21:21:32,123 INFO: Creating Detector SCA05 with (RA,DEC,PA) = (29.84327149478523,-30.119578781351112,0.0)
2024-04-05 21:21:32,124 INFO: Creating Detector SCA05 with offset (-0.15672850521477064,-0.11957878135111233)
2024-04-05 21:21:32,178 INFO: SCA05: (RA, DEC, PA) := (29.84327149478523, -30.119578781351112, 0.0), detected as (29.843271494

Output FITS file is /data/bwedig/STIPS/hlwas_sim_1_1.fits
Output Mosaic file is None
Observation Parameters are ['Instrument: WFI', 'Filters: F129', 'Pixel Scale: (0.110,0.110) arcsec/pix', 'Pivot Wavelength: 1.293 um', 'Background Type: Set to 0.15', 'Exposure Time: 1000.0s', 'Input Unit: counts/s', 'Added Flatfield Residual: False', 'Added Dark Current Residual: False', 'Added Cosmic Ray Residual: False', 'Added Readnoise: False', 'Added Poisson Noise: False', 'Detector X size: 4088', 'Detector Y size: 4088', 'Random Seed: 0']


2024-04-05 21:21:50,463 INFO: SCA04: (RA, DEC, PA) := (29.8433541331425, -29.972902151206657, 0.0), detected as (29.8433541331425, -29.972902151206657, 0.0)
2024-04-05 21:21:50,463 INFO: Detector SCA04 created
2024-04-05 21:21:50,464 INFO: Creating Detector SCA05 with (RA,DEC,PA) = (29.84327149478523,-30.119578781351112,0.0)
2024-04-05 21:21:50,464 INFO: Creating Detector SCA05 with offset (-0.15672850521477064,-0.11957878135111233)
2024-04-05 21:21:50,518 INFO: SCA05: (RA, DEC, PA) := (29.84327149478523, -30.119578781351112, 0.0), detected as (29.84327149478523, -30.119578781351112, 0.0)
2024-04-05 21:21:50,519 INFO: Detector SCA05 created
2024-04-05 21:21:50,519 INFO: Creating Detector SCA06 with (RA,DEC,PA) = (29.843451405244423,-30.250174749241296,0.0)
2024-04-05 21:21:50,520 INFO: Creating Detector SCA06 with offset (-0.15654859475557562,-0.2501747492412948)
2024-04-05 21:21:50,573 INFO: SCA06: (RA, DEC, PA) := (29.843451405244423, -30.250174749241296, 0.0), detected as (29.843451

Output FITS file is /data/bwedig/STIPS/hlwas_sim_1_2.fits
Output Mosaic file is None
Observation Parameters are ['Instrument: WFI', 'Filters: F184', 'Pixel Scale: (0.110,0.110) arcsec/pix', 'Pivot Wavelength: 1.842 um', 'Background Type: Set to 0.15', 'Exposure Time: 1000.0s', 'Input Unit: counts/s', 'Added Flatfield Residual: False', 'Added Dark Current Residual: False', 'Added Cosmic Ray Residual: False', 'Added Readnoise: False', 'Added Poisson Noise: False', 'Detector X size: 4088', 'Detector Y size: 4088', 'Random Seed: 0']


In [58]:
# import all FITS files
fits_files = glob(os.path.join(output_dir, f'{obs_prefix}_{obs_id}_*.fits'))
fits_files.sort()

for f in fits_files:
    pprint(fits.info(f))

Filename: /data/bwedig/STIPS/hlwas_sim_1_0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SCA01         1 ImageHDU       831   (4088, 4088)   float32   
  2  SCA02         1 ImageHDU        41   (4088, 4088)   float32   
  3  SCA03         1 ImageHDU        41   (4088, 4088)   float32   
  4  SCA04         1 ImageHDU        41   (4088, 4088)   float32   
  5  SCA05         1 ImageHDU        41   (4088, 4088)   float32   
  6  SCA06         1 ImageHDU        41   (4088, 4088)   float32   
  7  SCA07         1 ImageHDU        41   (4088, 4088)   float32   
  8  SCA08         1 ImageHDU        41   (4088, 4088)   float32   
  9  SCA09         1 ImageHDU        41   (4088, 4088)   float32   
 10  SCA10         1 ImageHDU        41   (4088, 4088)   float32   
 11  SCA11         1 ImageHDU        41   (4088, 4088)   float32   
 12  SCA12         1 ImageHDU        41   (4088, 4088)   float32   
 13  SCA13         1 Im

In [59]:
for f, band in zip(fits_files, bands):
    for detector in detector_list:
        image = fits.getdata(f, extname=detector)
        np.save(os.path.join(output_dir, f'{obs_prefix}_{band}_{detector}.npy'), image)

In [60]:
# # organize
# scene = {}
# for f, band in zip(fits_files, bands):
#     images = []
#     for detector in detector_list:
#         image = fits.getdata(f, extname=detector)
#         images.append(image)
#     scene[band] = images

# pickle_path = os.path.join(output_dir, 'scene.pkl')
# with open(pickle_path, 'ab') as results_file:
#     pickle.dump(scene, results_file)

In [61]:
# rgb_images = []
# for i, detector in tqdm(enumerate(detector_list)):
#     to_combine = []
#     for band in bands:
#         to_combine.append(scene[band][i])
#     min = np.min([scene['F106'][i], scene['F129'][i], scene['F184'][i]])
#     rgb = make_lupton_rgb(*to_combine, minimum=min, stretch=0.5, Q=4)
#     rgb_images.append(rgb)

# save_path = os.path.join(output_dir, 'hlwas_rgb.npy')
# np.save(save_path, rgb_images)

In [62]:
# plt.imshow(np.log10(scene['F106'][0]))
# plt.colorbar()
# plt.show()

In [63]:
# min = np.min([scene['F106'][0], scene['F129'][0], scene['F184'][0]])

# rgb_image = make_lupton_rgb(scene['F106'][0], scene['F129'][0], scene['F184'][0], minimum=min, stretch=1.5, Q=8)

# plt.imshow(rgb_image)
# plt.show()

In [64]:
stop = time.time()
execution_time = str(datetime.timedelta(seconds=round(stop - start)))
print(f'Execution time: {execution_time}')

Execution time: 0:01:09
